# Rs 8 0 1_ Hypo Test Inverter Original
An example of using the HypoTestInverterOriginal class




**Author:** Gregory Schott  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 03:23 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooConstVar.h"
#include "RooProdPdf.h"
#include "RooWorkspace.h"
#include "RooDataSet.h"
#include "RooPolynomial.h"
#include "RooAddPdf.h"
#include "RooExtendPdf.h"

#include "RooStats/HypoTestInverterOriginal.h"
#include "RooStats/HypoTestInverterResult.h"
#include "RooStats/HypoTestInverterPlot.h"
#include "RooStats/HybridCalculatorOriginal.h"

#include "TGraphErrors.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;
using namespace RooStats;

Prepare the model

In [3]:
RooRealVar lumi("lumi", "luminosity", 1);
RooRealVar r("r", "cross-section ratio", 3.74, 0, 50);
RooFormulaVar ns("ns", "1*r*lumi", RooArgList(lumi, r));
RooRealVar nb("nb", "background yield", 1);
RooRealVar x("x", "dummy observable", 0, 1);
RooConstVar p0(RooFit::RooConst(0));
RooPolynomial flatPdf("flatPdf", "flat PDF", x, p0);
RooAddPdf totPdf("totPdf", "S+B model", RooArgList(flatPdf, flatPdf), RooArgList(ns, nb));
RooExtendPdf bkgPdf("bkgPdf", "B-only model", flatPdf, nb);
RooDataSet *data = totPdf.generate(x, 1);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Prepare the calculator

In [4]:
HybridCalculatorOriginal myhc(*data, totPdf, bkgPdf, 0, 0);
myhc.SetTestStatistic(2);
myhc.SetNumberOfToys(1000);
myhc.UseNuisance(false);

Run the hypothesis-test inversion

In [5]:
HypoTestInverterOriginal myInverter(myhc, r);
myInverter.SetTestSize(0.10);
myInverter.UseCLs(true);

Myinverter.runfixedscan(5,1,6);
 scan for a 95% UL

In [6]:
myInverter.RunAutoScan(3., 5, myInverter.Size() / 2, 0.005);

Target matches upper limit: de-activate interpolation in HypoTestInverterResult
Running for r = 3
Test statistics has been evaluated for data
HybridCalculatorOriginal: run 1000 toy-MC experiments
with test statistics index: 2
....... toy number 0 / 1000
....... toy number 500 / 1000
computed: 0.34252
Running for r = 5
Test statistics has been evaluated for data
HybridCalculatorOriginal: run 1000 toy-MC experiments
with test statistics index: 2
....... toy number 0 / 1000
....... toy number 500 / 1000
computed: 0.0669145
The confidence level at both boundaries are both too large ( 0.34252 and 0.0669145
Run again with other boundaries or larger toy-MC statistics


Run an alternative autoscan algorithm
 myInverter.RunAutoScan(1,6,myInverter.Size()/2,0.005,1);
 myInverter.RunOnePoint(3.9);

In [7]:
HypoTestInverterResult *results = myInverter.GetInterval();

HypoTestInverterPlot myInverterPlot("myInverterPlot", "", results);
TGraphErrors *gr1 = myInverterPlot.MakePlot();
gr1->Draw("ALP");

double ulError = results->UpperLimitEstimatedError();

double upperLimit = results->UpperLimit();
std::cout << "The computed upper limit is: " << upperLimit << std::endl;
std::cout << "an estimated error on this upper limit is: " << ulError << std::endl;

The computed upper limit is: 5
an estimated error on this upper limit is: 0


Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


Expected result: 4.10

Draw all canvases 

In [8]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()